In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
#import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import ensemble
import time
import pickle
import random
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import OneHotEncoder
import re
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Merge
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional,concatenate
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
import re
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 

In [6]:
train=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/train.csv')
test=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/test.csv')
sample_submission=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/sample_submission.csv')

#Test

In [7]:
#test.head()

In [8]:
train_copy=train.drop(['Complaint-Status'],axis=1)

In [9]:
#Joining Available Columns in Test and Train for Pre-processing
join=train_copy.append(test,sort=False)

In [10]:
#Date Time
join['Date-received'] =  pd.to_datetime(join['Date-received'], format='%m/%d/%Y')
join['Date-sent-to-company'] =  pd.to_datetime(join['Date-sent-to-company'], format='%m/%d/%Y')

In [11]:
join['diff_time']=join['Date-sent-to-company']-join['Date-received']
join['diff_time']=join['diff_time'].apply(lambda x:x.days )

#Imputing Missing Values for two columns 

Making a Separate category for missing

In [12]:
#Company Respose
join['Company-response'].fillna(value='No response',inplace =True)

#Missing Dispute-- No Dispute
join['Consumer-disputes'].fillna(value=join['Consumer-disputes'].value_counts().index[0],inplace =True)

In [13]:
count_trans=len(join['Transaction-Type'].unique())
print count_trans

18


In [14]:
count_complain_reason=len(join['Complaint-reason'].unique())
print count_complain_reason

152


In [15]:
count_company_response=len(join['Company-response'].unique())
print count_company_response

11


In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [17]:
diff_time=np.array(join['diff_time']).reshape(-1,1)

In [18]:
enc_time=scaler.fit_transform(diff_time)

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Processing The categorical Variables

In [19]:
#1.Label Encoding the Categorical Varaible----TransactionType
le1= preprocessing.LabelEncoder()
enc_trans_type=le1.fit_transform(join['Transaction-Type'])

In [20]:
#2.Label Encoding the Categorical Varaible----Complaint-reason
le2= preprocessing.LabelEncoder()
enc_complain_reason=le2.fit_transform(join['Complaint-reason'])

In [21]:
#3.Label Encoding the Categorical Varaible----Company-response
le3= preprocessing.LabelEncoder()
enc_comp_response=le3.fit_transform(join['Company-response'])

In [22]:
#4.Label Encoding the Categorical Varaible----Consumer-disputes
le4= preprocessing.LabelEncoder()
enc_cons_dispute=le4.fit_transform(join['Consumer-disputes'])

In [23]:
#Response 
le5= preprocessing.LabelEncoder()
y_cat=le5.fit_transform(train['Complaint-Status'])

In [24]:
from sklearn.preprocessing import StandardScaler


In [25]:
from keras.utils import to_categorical
y_enc=to_categorical(y_cat,num_classes=5)

Processing the Text Data Column----> Consumer-complaint-summary

In [26]:
join['Consumer-complaint-summary'] = join['Consumer-complaint-summary'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))

In [27]:
join['Consumer-complaint-summary'] = join['Consumer-complaint-summary'].apply(lambda x: str(x))

In [28]:
join['Consumer-complaint-summary']=join['Consumer-complaint-summary'].str.replace('XX','')

In [29]:
join['Consumer-complaint-summary']=join['Consumer-complaint-summary'].apply(lambda x:x.lower())

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english',max_df=1.0, min_df=1, max_features=50000)
tf_feat = vectorizer.fit_transform(join['Consumer-complaint-summary'])

In [113]:
from scipy.sparse import csr_matrix
tf_idf_arr= tf_feat.toarray()

#Inputs for our Data

In [114]:
#Transaction
enc_trans_type_train=enc_trans_type[0:43266]
enc_trans_type_test=enc_trans_type[43266:]

In [115]:
#Complaint reason
enc_complain_reason_train=enc_complain_reason[0:43266]
enc_complain_reason_test=enc_complain_reason[43266:]

In [116]:
enc_comp_response_train=enc_comp_response[0:43266]
enc_comp_response_test= enc_comp_response[43266:]

In [117]:
enc_cons_dispute_train=enc_cons_dispute[0:43266]
enc_cons_dispute_test= enc_cons_dispute[43266:]

In [118]:
enc_time_train=enc_time[0:43266]
enc_time_test=enc_time[43266:]

In [119]:
tf_idf_arr_train=tf_idf_arr[0:43266]
tf_idf_arr_test=tf_idf_arr[43266:]

In [120]:
def model_deep():
    
    #Defining the input-----> Transaction
    inputs1 = Input(shape=(1,))
    layer_transaction = Embedding(count_trans ,5,input_length=1,trainable=True)(inputs1)
    layer_transaction = Flatten()(layer_transaction)
    
    #Defining the input-----> Complaint-reason
    inputs2 = Input(shape=(1,))
    layer_complain_reason = Embedding(count_complain_reason ,50,input_length=1,trainable=True)(inputs2)
    layer_complain_reason = Flatten()(layer_complain_reason)
    
    #Defining the input-----> Complaint-reason
    inputs3 = Input(shape=(1,))
    layer_company_response = Embedding(count_company_response ,3,input_length=1,trainable=True)(inputs3)
    layer_company_response = Flatten()(layer_company_response)
    
    #Defining the input-----> Consumer-disputes
    inputs4 = Input(shape=(1,))
    layer_consumer_disputes = Embedding(2 ,1,input_length=1,trainable=True)(inputs4)
    layer_consumer_disputes = Flatten()(layer_consumer_disputes)
    
    #Defining the input ----> Time
    inputs5 = Input(shape=(1,))
    
    #Defining the input-----> Consumer-complaint-summary
    inputs6 = Input(shape=(50000,))
    
    #Merge,Cocatenating the inputes
    layer_merge= concatenate([layer_transaction, layer_complain_reason, 
                              layer_company_response,layer_consumer_disputes,inputs5,inputs6])
    
    #Dense Layers
    layer_dense = Dense(5000, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    
    #Dense Layers
    layer_dense = Dense(500, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
   

    #Output Layer
    probabilities = Dense(5,activation='softmax')(layer_dense)

    model = Model(inputs=[inputs1,inputs2,inputs3,inputs4,inputs5,inputs6],outputs=probabilities)
    return model


In [121]:
model=model_deep()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_43 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_44 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_45 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_46 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_

In [122]:
model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr=0.001), metrics = ["accuracy"])

In [123]:
#Deep Learning Model
history = model.fit([enc_trans_type_train,
                     enc_complain_reason_train,
                     enc_comp_response_train,
                     enc_cons_dispute_train,
                     enc_time_train,
                     tf_idf_arr_train] ,y_enc, batch_size = 512, epochs = 4, validation_split=0.1, 
                     verbose = 1)
                    
                    #class_weight = {0:0.25 ,1:0.01 ,2:0.25 ,3:0.24 ,4:0.25 })



Train on 38939 samples, validate on 4327 samples
Epoch 1/4
38939/38939 [==============================] - 134s 3ms/step - loss: 0.8178 - acc: 0.7834 - val_loss: 0.6401 - val_acc: 0.7959
Epoch 2/4
38939/38939 [==============================] - 122s 3ms/step - loss: 0.5970 - acc: 0.7978 - val_loss: 0.6041 - val_acc: 0.7939
Epoch 3/4
38939/38939 [==============================] - 145s 4ms/step - loss: 0.5295 - acc: 0.8084 - val_loss: 0.5920 - val_acc: 0.7964
Epoch 4/4
38939/38939 [==============================] - 149s 4ms/step - loss: 0.4662 - acc: 0.8271 - val_loss: 0.5981 - val_acc: 0.7950


In [107]:
#Tets
y_prob = model.predict([enc_trans_type_test,enc_complain_reason_test,
                     enc_comp_response_test,enc_cons_dispute_test,enc_time_test,tf_idf_arr_test]) 

#Pred
y_classes = y_prob.argmax(axis=-1)

In [108]:
y_pred=list(le5.inverse_transform(y_classes))

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [109]:
#y_pred

In [110]:
submission=pd.DataFrame()
submission['Complaint-ID']=test['Complaint-ID']
submission['Complaint-Status']=y_pred

In [111]:
submission.to_csv(path_or_buf="/Users/s0c02nj/Desktop/Submission_deep.csv", encoding='utf-8',index=False)

In [91]:
np.unique(y_cat)

array([0, 1, 2, 3, 4])

In [95]:
pd.Series(y_cat).value_counts()

1    34300
3     5018
2     2818
0      809
4      321
dtype: int64

In [92]:
train['Complaint-Status'].value_counts()

Closed with explanation            34300
Closed with non-monetary relief     5018
Closed with monetary relief         2818
Closed                               809
Untimely response                    321
Name: Complaint-Status, dtype: int64